# Dream Destination Finder with CrewAI and Amazon Bedrock

In this notebook, we will explore how to use the CrewAI framework with Amazon Bedrock to build an intelligent agent that can find dream travel destinations based on user preferences. The agent will utilize a large language model (LLM) and web search capabilities to research and recommend destinations that match the user's description.

## What's CrewAI:
CrewAI is one of the leading open-source Python frameworks designed to help developers create and manage multi-agent AI systems.

<img src="assets/crewai_diagram.png">

Diagram Representation of CrewAI architecture

__!pip install boto3 botocore crewai==0.70.1 crewai_tools==0.12.1 duckduckgo-search langchain-community -q__

We start by importing the necessary modules from the crewai and crewai_tools packages.

#### Configuring AWS Credentials:
Before using Amazon Bedrock, ensure that your AWS credentials are configured correctly. You can set them up using the AWS CLI or by setting environment variables. For this notebook, we’ll assume that the credentials are already configured.

To use bedrock we will use [__CrewAI__ __LLM__ api](https://docs.crewai.com/how-to/llm-connections#supported-providers) 

In [23]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### Define web-search tool:

In [24]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Configuring the LLM

We will use Anthropic’s Claude-3 model via Amazon Bedrock as our LLM. CrewAI uses LiteLLM under the hood to interact with different LLM providers.


In [25]:
# Configure the LLM
llm = LLM(model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0")

### Defining the Agent

We will create an agent with the role of a “Travel Destination Researcher.” This agent will be responsible for finding destinations that match the user’s travel preferences.

In [46]:
# Define the Agent
travel_agent = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations matching user preferences',
    backstory="You are an experienced travel agent specializing in personalized travel recommendations.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

### Defining the Task

We need to specify the task that the agent will perform. The task includes a description, expected output, and is assigned to the agent we just created.

In [89]:
# Define the Task
task = Task(
    description="Based on the user's travel preferences: {preferences}, research and recommend suitable travel destinations.",
    expected_output="A list of recommended destinations with brief descriptions.",
    agent=travel_agent
)

### Creating the Crew

A crew is a team of agents working together to achieve a common goal. In this case, we have only one agent, but the framework allows for scalability.


In [90]:
# Create the Crew
crew = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
)

2024-10-15 11:34:01,412 - 8603045696 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


### Executing the Workflow

Now, we can execute the crew with the user’s travel preferences as input.

In [91]:
# User input for travel preferences
user_input = {
    "preferences": "I want a tropical beach vacation with great snorkeling and vibrant nightlife."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To provide suitable travel destination recommendations based on the user's preferences for a tropical beach vacation with great snorkeling and vibrant nightlife, I need to gather information on destinations that meet those criteria.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach destinations with great snorkeling and nightlife\"}"
## Tool Output: 
19. Boracay, Philippines. Boracay, with its powdery white sand and azure waters, is often considered one of the top tropical islands in the world. Visitors can enjoy water sports on White Beach, explore the quieter Puka Shell Beach, or witness spectacular sunsets from the island's western shore. While Los Cabos is known for chop

#### As the crew executes, CrewAI will:

•	Decompose the task into actions using ReAct (Reasoning and Act), optionally using the tools assigned to the agent.

•	Make multiple calls to Amazon Bedrock to complete each step from the previous phase.

In [92]:
from IPython.display import Markdown

In [93]:
Markdown(result.raw)

1. Boracay, Philippines: This tropical island is renowned for its powdery white sand beaches like White Beach, excellent snorkeling opportunities, and vibrant nightlife scene. Visitors can enjoy water sports, explore quieter beaches like Puka Shell Beach, and experience spectacular sunsets.

2. Cabo San Lucas and Todos Santos, Mexico: While Cabo San Lucas is known for its lively nightlife and stunning beaches, the nearby town of Todos Santos offers great snorkeling spots like Playa Los Cerritos and Punta Lobos. This combination provides both vibrant nightlife and excellent snorkeling opportunities.

3. Ogasawara Islands, Japan: Located south of Tokyo, these remote islands offer the chance to snorkel with wild Indo-Pacific bottlenose dolphins in their natural habitat. While the waters can be choppy, the experience of swimming with dolphins in a tropical setting is truly unique.

4. Devil's Crown, Galápagos Islands, Ecuador: This submerged volcanic cone near Floreana Island is celebrated for its abundant marine life. Snorkelers can encounter sea lions, turtles, and various fish species, making it an ideal destination for those seeking an exceptional snorkeling experience in a tropical setting.

5. N'Gouja Beach, Mayotte: This French territory in the Indian Ocean boasts a wide sandy beach, spectacular coral reef, and superb biodiversity, making it a rewarding destination for snorkeling. N'Gouja Beach offers a tropical paradise-like setting with excellent snorkeling opportunities.

### Adding Memory to the Agent
CrewAI supports [several memory types](https://docs.crewai.com/concepts/memory#implementing-memory-in-your-crew), which help agents remember and learn from past interactions. In this case, we’ll enable short-term memory using Amazon Bedrock’s embedding model.

In [94]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v2:0",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

2024-10-15 11:34:12,282 - 8603045696 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [95]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To find suitable travel destinations that match the user's preferences of a tropical beach vacation with great snorkeling and vibrant nightlife, I will perform a series of searches to gather relevant information.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach destinations with great snorkeling\"}"
## Tool Output: 
Located 124 miles south of Tokyo, it's one of the few places in the world you can snorkel with wild Indo-Pacific bottlenose dolphins. The waters are choppy off this far-flung island, so its best ... 15. Snorkeling in Carlisle Bay, Barbados. Barbados has some of the best snorkeling spots in the Caribbean, especially in Carlisle Bay. The Bay has a marine park wher

In [103]:
Markdown(result_with_memory.raw)

Here are some recommended destinations for a tropical beach vacation with great snorkeling and vibrant nightlife:

1. Boracay, Philippines - Known for its stunning white sand beaches, crystal clear waters perfect for snorkeling, and a lively nightlife scene with beach parties and bars. 

2. Isla Mujeres, Mexico - This island off the coast of Cancun boasts excellent snorkeling opportunities to explore the vibrant marine life and coral reefs in the Mexican Caribbean, as well as a bustling downtown area with restaurants, bars and nightlife.

3. Phuket, Thailand - Offering world-class snorkeling and diving spots like the Similan Islands, as well as the lively Patong Beach area filled with nightclubs, bars, and entertainment venues.

4. Bali, Indonesia - With beaches like Sanur and Nusa Dua providing access to colorful reefs for snorkeling, and areas like Seminyak and Kuta known for their vibrant beach clubs, bars and parties.

5. Barbados - Renowned for its snorkeling hotspots like Carlisle Bay Marine Park, this Caribbean island also features lively nightlife in areas such as St. Lawrence Gap with bars, restaurants and entertainment.

6. Zanzibar, Tanzania - In addition to its pristine beaches ideal for snorkeling and abundant marine life, Zanzibar is celebrated for its vibrant culture, historical Stone Town and energetic nightlife scene.

7. Puerto Vallarta, Mexico - With excellent snorkeling opportunities along the Banderas Bay, Puerto Vallarta also offers a diverse nightlife with bars, clubs and entertainment concentrated in areas like the Malecón and Romantic Zone.

### Integrating Retrieval-Augmented Generation (RAG) with Amazon Bedrock Knowledge Base
In this section, we will enhance our dream destination finder agent by incorporating Retrieval-Augmented Generation (RAG) using Amazon Bedrock’s Knowledge Base. This will allow our agent to access up-to-date and domain-specific travel information, improving the accuracy and relevance of its recommendations.



#### What is Retrieval-Augmented Generation (RAG)?

RAG is a technique that combines the capabilities of large language models (LLMs) with a retrieval mechanism to fetch relevant information from external data sources. By integrating RAG, our agent can retrieve the most recent and specific information from a knowledge base, overcoming the limitations of LLMs that may have outdated or insufficient data.

Setting Up Amazon Bedrock Knowledge Base

Before we proceed, ensure you have access to Amazon Bedrock and the necessary permissions to create and manage knowledge bases.

* Step 1: Prepare Your Data
* Step 2: Create a Knowledge Base in Amazon Bedrock
* Step 3: Note the Knowledge Base ID

After the knowledge base is created, note down its Knowledge Base ID (kb_id), which will be used in our code.

<img src="assets/KB-pannel.png">

Updating the Agent to Use RAG with CrewAI

We will modify our agent to include a custom tool that queries the Amazon Bedrock Knowledge Base. This allows the agent to retrieve up-to-date information during its reasoning process.

In [130]:
import boto3
# Initialize the Bedrock client
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name="{YOUR-REGION}")

### Knowledge Base Tool Set up:
Using the __kb id__, __model arn__ (either foundational or custom) we can leverage Amazons Knowledge Bases. In this example the question will also be broken down using __orchestrationConfiguration__ settings.

In [248]:
@tool("TravelExpertSearchEngine")
def query_knowledge_base(question: str) -> str:
    """Queries the Amazon Bedrock Knowledge Base for travel-related information."""
    kb_id = "XXXX"  # Replace with your Knowledge Base ID
    model_id = "foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"   # Use an available model in Bedrock
    model_arn = f'arn:aws:bedrock:YOUR-REGION::{model_id}'

    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={'text': question},
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration" : {'knowledgeBaseId': kb_id,
                                    'modelArn': model_arn,
                                    'orchestrationConfiguration': {
                                        'queryTransformationConfiguration': {
                                            'type': 'QUERY_DECOMPOSITION'
                                        }
                                    }
                                            }
        }
    )
    try:
        return str({"Results": response['output']['text'], "Citations": response['citations'][0]})
    except KeyError:
        return "No data available"



### Update the Agent with the New Tool
We will update our agent to include the TravelExpert tool.

In [251]:
# Configure the LLM
llm = LLM(model="bedrock/anthropic.claude-3-sonnet-20240229-v1:0")

# Update the Agent
agent_with_rag = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations in the USA, first think about cities matching user preferences and then use information from the search engine, nothing else.',
    backstory="""You are an experienced travel agent specializing in personalized travel recommendations. 
                 Your approach is as follows: 
                 Deduce which regions within the USA will have those activities listed by the user.
                 List major cities within that region
                 Only then use the tool provided to look up information, look up should be done by passing city highlights and activities.
              """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[query_knowledge_base],  # Include the RAG tool
    max_iter=5
)


### Update the task and set up the Crew

In [252]:
# Define the Task
task_with_rag = Task(
    description="Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: {preferences}",
    expected_output="A place where they can travel to along with recommendations on what to see and do while there.",
    agent=agent_with_rag
)


# Create the Crew
crew_with_rag = Crew(
    agents=[agent_with_rag],
    tasks=[task_with_rag],
    verbose=True,
)

2024-10-16 12:47:21,395 - 8603045696 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [253]:
# User input for travel preferences
user_input = {
    "preferences": "Where can I go for cowboy vibes, watch a rodeo, and a museum or two?"
}

# Execute the Crew
result_with_rag = crew_with_rag.kickoff(inputs=user_input)


# Agent: Travel Destination Researcher
## Task: Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: Where can I go for cowboy vibes, watch a rodeo, and a museum or two?


# Agent: Travel Destination Researcher
## Thought: Thought: To recommend destinations for cowboy vibes, rodeos, and museums, I should first consider regions of the USA that are known for their cowboy culture and western heritage. The southwestern states and parts of the Great Plains seem like a good starting point. I should also think about major cities in those regions that would likely have rodeo events and museums related to western and cowboy history.
## Using tool: TravelExpertSearchEngine
## Tool Input: 
"{\"question\": \"Cities in the southwestern US or Great Plains known for cowboy culture, rodeos, and western history museums\"}"
## Tool Output: 
{'Results': 'Dall

### Display the results

In [256]:
# Display the result
Markdown(result_with_rag.raw)

Based on your interests in cowboy culture, rodeos, and museums, I would recommend traveling to the Dallas, Texas area. Dallas has a thriving honky-tonk bar scene in the Deep Ellum neighborhood that celebrates its cowboy heritage and allows you to experience authentic cowboy vibes. 

While major rodeo events were not specifically mentioned in the search results, Dallas likely hosts or has access to rodeos given its location and ties to western culture. The city also has the Sixth Floor Museum which chronicles the life and assassination of President John F. Kennedy, providing an opportunity to learn about an important event in American history.

Some potential activities in the Dallas area could include:

- Visiting authentic honky-tonk bars like The Lil' Red Saloon in Deep Ellum to experience live country music and a cowboy atmosphere
- Seeing exhibits on the history of the American West at the Sixth Floor Museum 
- Attending rodeo events or visiting the Fort Worth Stockyards historic district just outside Dallas to watch cattle drives and see cowboy culture
- Exploring outdoor activities like horseback riding at nearby ranches or state parks
- Checking out the Nasher Sculpture Center or other museums in the Dallas Arts District

With its blend of cowboy heritage, history museums, and proximity to rodeos and outdoor adventures, the Dallas metro area could make an excellent travel destination aligning with your stated interests. Let me know if you need any other details to plan your cowboy/rodeo themed travels!